# CIFAR-10 데이터셋 분류(Skip Conncection, Dense Connection, Channel Attention)

### 폴더 경로 저장

In [ ]:
datasetPath = "./drive/MyDrive/dataset/"
parameterPath = "./drive/MyDrive/parameters/"

### 패키지 선언

In [ ]:

import torch
import torch.nn as nn
import torchvision.datasets as dataset
import torchvision.transforms as transform
from torch.utils.data import DataLoader
import numpy as np

### 데이터셋 선언 (CIFAR10)

In [ ]:
# Training dataset 다운로드
cifar10_train = dataset.CIFAR10(root = datasetPath, # 데이터셋이 저장된 위치
                            train = True,
                            transform = transform.ToTensor(),
                            download = True)
# Testing dataset 다운로드
cifar10_test = dataset.CIFAR10(root = datasetPath,
                            train = False,
                            transform = transform.ToTensor(),
                            download = True)

  0%|          | 0/170498071 [00:00<?, ?it/s]

Extracting ./drive/MyDrive/dataset/cifar-10-python.tar.gz to ./drive/MyDrive/dataset/
Files already downloaded and verified


## Skip Connection

In [ ]:
#CNN 모델 (VGG) 정의
class VGG_SKIP (nn.Module):
    def __init__(self): # 신경망 구성요소 정의
        super(VGG_SKIP, self).__init__()
        self.conv1_1 = nn.Conv2d(in_channels=3, out_channels=16, kernel_size=3, padding=1)
        self.conv1_2 = nn.Conv2d(in_channels=16, out_channels=32, kernel_size=3, padding=1)
               
        self.conv2_1 = nn.Conv2d(in_channels=32, out_channels=32, kernel_size=3, padding=1)
        self.conv2_2 = nn.Conv2d(in_channels=32, out_channels=64, kernel_size=3, padding=1)
        
        self.conv3_1 = nn.Conv2d(in_channels=64, out_channels=128, kernel_size=3, padding=1)
        self.conv3_2 = nn.Conv2d(in_channels=128, out_channels=256, kernel_size=3, padding=1)

        # Skip Connection을 위한 Conv, layer
        self.conv_skip1 = nn.Conv2d(in_channels=3, out_channels=32, kernel_size=3, padding=1)
        self.conv_skip2 = nn.Conv2d(in_channels=32, out_channels=64, kernel_size=3, padding=1)
        self.conv_skip3 = nn.Conv2d(in_channels=64, out_channels=256, kernel_size=3, padding=1)

        self.fc1 = nn.Linear(in_features=4096, out_features=512)
        self.fc2 = nn.Linear(in_features=512, out_features=256)
        self.fc3 = nn.Linear(in_features=256, out_features=10)
        
        self.relu = nn.ReLU()
        self.avgPool2d = nn.AvgPool2d(kernel_size=2, stride=2)

    def forward(self,x):

        input_feature1 = x  # skip 입력을 위한 input 저장
        out = self.relu(self.conv1_1(x))
        out = self.relu(self.conv1_2(out))
        input_skip1 = self.relu(self.conv_skip1(input_feature1))  # skip 입력을 위한 con layer 적용
        out = torch.add(out, input_skip1) # skip connection 적용
        out = self.avgPool2d(out)

        input_feature2 = out  # skip 입력을 위한 input 저장
        out = self.relu(self.conv2_1(out))
        out = self.relu(self.conv2_2(out))
        input_skip2 = self.relu(self.conv_skip2(input_feature2))  # skip 입력을 위한 con layer 적용
        out = torch.add(out, input_skip2) # skip connection 적용
        out = self.avgPool2d(out)

        input_feature3 = out  # skip 입력을 위한 input 저장
        out = self.relu(self.conv3_1(out))
        out = self.relu(self.conv3_2(out))
        input_skip3 = self.relu(self.conv_skip3(input_feature1))  # skip 입력을 위한 con layer 적용
        out = torch.add(out, input_skip3) # skip connection 적용
        out = self.avgPool2d(out)

        out = out.view(-1, 4096) # feature map 평탄화

        out = self.relu(self.fc1(out))
        out = self.relu(self.fc2(out))
        out = self.fc3(out)
        return out

In [ ]:
#Hyper-parameters 지정
training_epoch = 20
batch_size = 100
learning_rate = 0.1
loss_function = nn.CrossEntropyLoss()
network = VGG_SKIP()
optimizer = torch.optim.SGD(network.parameters(), lr = learning_rate)

data_loader = DataLoader(dataset=cifar10_train,
                         batch_size=batch_size,
                         shuffle=True,
                         drop_last=True)

In [ ]:
#Trainin loop w/ GPU
#- Colab 상단 런타임 - 런타임 유형 변경 - 하드웨어 가속기 에서 GPU 선택 - 저장
#- !nvidia-smi 명령어로 GPU 할당 여부 확인
#- network parameter 및 data 유형을 cuda:0으로 설정
network.train()
network = network.to('cuda:0')
for epoch in range(training_epoch):
    avg_cost = 0
    total_batch = len(data_loader)

    for img, label in data_loader:
        
        img = img.to('cuda:0')
        label = label.to('cuda:0')
        pred = network(img)
        loss = loss_function(pred, label)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        avg_cost += loss / total_batch

    print('Epoch: %d Loss = %f'%(epoch+1, avg_cost))

print('Learning finished')

RuntimeError: ignored

In [ ]:
#Weight parameter 저장
torch.save(network.state_dict(), parameterPath+"vgg_cifar10.pth")

In [ ]:
#분류 성능 확인
network.eval()
network = network.to('cpu')
img_test = torch.tensor(np.transpose(cifar10_test.data , (0, 3, 1, 2))) /255.
label_test = torch.tensor(cifar10_test.targets)

with torch.no_grad(): # test에서는 기울기 계산 제외
    prediction = network(img_test) # 전체 test data를 한번에 계산

correct_prediction = torch.argmax(prediction, 1) == label_test
accuracy = correct_prediction.float().mean()
print('Accuracy:', accuracy.item())

## Dence Connection

In [ ]:
#CNN 모델 (VGG) 정의
class VGG_DENSE (nn.Module):
    def __init__(self): # 신경망 구성요소 정의
        super(VGG_DENSE, self).__init__()
        self.conv1_1 = nn.Conv2d(in_channels=3, out_channels=16, kernel_size=3, padding=1)
        self.conv1_2 = nn.Conv2d(in_channels=16, out_channels=32, kernel_size=3, padding=1)
               
        self.conv2_1 = nn.Conv2d(in_channels=35, out_channels=32, kernel_size=3, padding=1)
        self.conv2_2 = nn.Conv2d(in_channels=32, out_channels=64, kernel_size=3, padding=1)
        
        self.conv3_1 = nn.Conv2d(in_channels=99, out_channels=128, kernel_size=3, padding=1)
        self.conv3_2 = nn.Conv2d(in_channels=128, out_channels=256, kernel_size=3, padding=1)
        
        self.fc1 = nn.Linear(in_features=4096, out_features=512)
        self.fc2 = nn.Linear(in_features=512, out_features=256)
        self.fc3 = nn.Linear(in_features=256, out_features=10)
        
        self.relu = nn.ReLU()
        self.avgPool2d = nn.AvgPool2d(kernel_size=2, stride=2)

    def forward(self,x):

        out1 = self.relu(self.conv1_1(x))
        out1 = self.relu(self.conv1_2(out1))
        out1 = torch.cat([x, out1], dim=1)
        out1 = self.avgPool2d(out1)

        out2 = self.relu(self.conv2_1(out1))
        out2 = self.relu(self.conv2_2(out2))
        out2 = torch.cat([out1, out2], dim=1)
        out2 = self.avgPool2d(out2)

        out3 = self.relu(self.conv3_1(out2))
        out3 = self.relu(self.conv3_2(out3))
        #out3 = torch.cat([out2, out3], dim=1)
        out = self.avgPool2d(out3)

        out = out.view(-1, 4096) # feature map 평탄화

        out = self.relu(self.fc1(out))
        out = self.relu(self.fc2(out))
        out = self.fc3(out)
        return out

In [ ]:
#Trainin loop w/ GPU
#- Colab 상단 런타임 - 런타임 유형 변경 - 하드웨어 가속기 에서 GPU 선택 - 저장
#- !nvidia-smi 명령어로 GPU 할당 여부 확인
#- network parameter 및 data 유형을 cuda:0으로 설정
network.train()
network = network.to('cuda:0')
for epoch in range(training_epoch):
    avg_cost = 0
    total_batch = len(data_loader)

    for img, label in data_loader:
        
        img = img.to('cuda:0')
        label = label.to('cuda:0')
        pred = network(img)
        loss = loss_function(pred, label)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        avg_cost += loss / total_batch

    print('Epoch: %d Loss = %f'%(epoch+1, avg_cost))

print('Learning finished')

In [ ]:
#Weight parameter 저장
torch.save(network.state_dict(), parameterPath+"vgg_cifar10.pth

In [ ]:
#분류 성능 확인
network.eval()
network = network.to('cpu')
img_test = torch.tensor(np.transpose(cifar10_test.data , (0, 3, 1, 2))) /255.
label_test = torch.tensor(cifar10_test.targets)

with torch.no_grad(): # test에서는 기울기 계산 제외
    prediction = network(img_test) # 전체 test data를 한번에 계산

correct_prediction = torch.argmax(prediction, 1) == label_test
accuracy = correct_prediction.float().mean()
print('Accuracy:', accuracy.item())

## Channel_Attention

In [ ]:
#CNN 모델 (VGG) 정의
class VGG_CA (nn.Module):
    def __init__(self): # 신경망 구성요소 정의
        super(VGG_CA, self).__init__()
        self.conv1_1 = nn.Conv2d(in_channels=3, out_channels=16, kernel_size=3, padding=1)
        self.conv1_2 = nn.Conv2d(in_channels=16, out_channels=32, kernel_size=3, padding=1)
               
        self.conv2_1 = nn.Conv2d(in_channels=32, out_channels=32, kernel_size=3, padding=1)
        self.conv2_2 = nn.Conv2d(in_channels=32, out_channels=64, kernel_size=3, padding=1)
        
        self.conv3_1 = nn.Conv2d(in_channels=64, out_channels=128, kernel_size=3, padding=1)
        self.conv3_2 = nn.Conv2d(in_channels=128, out_channels=256, kernel_size=3, padding=1)
        
        self.fc1 = nn.Linear(in_features=4096, out_features=512)
        self.fc2 = nn.Linear(in_features=512, out_features=256)
        self.fc3 = nn.Linear(in_features=256, out_features=10)


        # Channel Attention
        self.adaptiveAvgPool2d = nn.AdaptiveAvgPool2d((1, 1)) # Global average pooling
        self.caconv1 = nn.Conv2d(in_channels=64, out_channels=64, kernel_size=1)
        self.caconv2 = nn.Conv2d(in_channels=64, out_channels=64, kernel_size=1)
        self.sigmoid = nn.Sigmoid()

        self.relu = nn.ReLU()
        self.avgPool2d = nn.AvgPool2d(kernel_size=2, stride=2)

    def forward(self,x):

        out = self.relu(self.conv1_1(x))
        out = self.relu(self.conv1_2(out))
        out = self.avgPool2d(out)

        out = self.relu(self.conv2_1(out))
        out = self.relu(self.conv2_2(out))
        
        ## Channel attention 적용
        caout = self.adaptiveAvgPool2d(out)
        caout = self.relu(self.caconv1(caout))
        caout = self.sigmoid(self.caconv2(caout))
        CA_map = caout.expand_as(out)
        out = out * CA_map
        
        out = self.avgPool2d(out)

        out = self.relu(self.conv3_1(out))
        out = self.relu(self.conv3_2(out))
        out = self.avgPool2d(out)

        out = out.view(-1, 4096) # feature map 평탄화

        out = self.relu(self.fc1(out))
        out = self.relu(self.fc2(out))
        out = self.fc3(out)
        return out

In [ ]:
#Trainin loop w/ GPU
#- Colab 상단 런타임 - 런타임 유형 변경 - 하드웨어 가속기 에서 GPU 선택 - 저장
#- !nvidia-smi 명령어로 GPU 할당 여부 확인
#- network parameter 및 data 유형을 cuda:0으로 설정
network.train()
network = network.to('cuda:0')
for epoch in range(training_epoch):
    avg_cost = 0
    total_batch = len(data_loader)

    for img, label in data_loader:
        
        img = img.to('cuda:0')
        label = label.to('cuda:0')
        pred = network(img)
        loss = loss_function(pred, label)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        avg_cost += loss / total_batch

    print('Epoch: %d Loss = %f'%(epoch+1, avg_cost))

print('Learning finished')

In [ ]:
#Weight parameter 저장
torch.save(network.state_dict(), parameterPath+"vgg_cifar10.pth")


In [ ]:
#분류 성능 확인
network.eval()
network = network.to('cpu')
img_test = torch.tensor(np.transpose(cifar10_test.data , (0, 3, 1, 2))) /255.
label_test = torch.tensor(cifar10_test.targets)

with torch.no_grad(): # test에서는 기울기 계산 제외
    prediction = network(img_test) # 전체 test data를 한번에 계산

correct_prediction = torch.argmax(prediction, 1) == label_test
accuracy = correct_prediction.float().mean()
print('Accuracy:', accuracy.item())